In [1]:
import xml.etree.cElementTree as ET
import random

In [2]:
import xml.etree.cElementTree as ET    #deal with the head

target_file = 'TEST/一首簡單的歌_主旋律.musicxml'
output_file = 'TEST/output/new_test_accompany.xml'



tree = ET.parse(target_file)
root = tree.getroot()
childs = root.getchildren()



for index, i in enumerate (childs):
    if i.tag=='part-list':
        new_part_list = childs[index]

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.
  # Remove the CWD from sys.path while we load stuff.


In [3]:
new_part1 = root.find('part')
new_part2 = ET.Element('part')
new_part2.set('id', 'C1')


new_root = ET.Element(root.tag)
new_root.set('version', '3.1')
new_root.append(new_part_list)


new_root.append(new_part1)
new_root.append(new_part2)


In [4]:
new_fifth = new_part1[0].find('attributes').find('key').find('fifths').text
print(new_fifth)


-1


In [5]:
new_volume = float(new_part_list[0].find('midi-instrument').find('volume').text)-32.5


In [6]:
from math import pow

fifth={0:0, -1:5, -2:-2, -3:3, -4: -4, -5:1, -6:-6,\
      1:-5, 2:2, 3:-3, 4:4, 5:-1, 6:6}
chord=[[0, 4, 7], [2, 5, 9], [4, 7, 11],[5,9,12], \
       [7,11,14], [-3,0,4], [-1,2,6 ]] #C,Dm,Em,F,G,Am,Bm
for index1, i in enumerate(chord):
    for index2, j in enumerate(i):
        chord[index1][index2]+=int(fifth[int(new_fifth)])

tempo_open=[['half', 0, [1,3],[0,0]],['eighth', 0, [1], [0]],['quarter', 1, [1,1], [0,1]]]
tempo_run=[[['quarter', 0, [1,2,3], [0,0,0]], ['quarter', 0, [1,3], [0,0]],['quarter', 0, [1,2,3], [0,0,0]], ['quarter', 0, [1,3], [0,0]]],\
           [['quarter', 0, [1,2,3], [0,0,0]],['eighth', 0, [2], [0]],['eighth', 0, [3], [0]],['eighth', 0, [2], [0]],['eighth', 0, [1], [0]],['quarter', 0, [1,3], [0,0]]],\
           [['quarter', 0, [1,2,3], [0,0,0]],['eighth', 0, [2], [0]],['eighth', 0, [3], [0]],['quarter', 0, [2], [0]],['quarter', 0, [1,3], [0,0]]],\
          [['eighth', 0, [1,2,3],[0,0,0]],['eighth', 0, [1,2],[0,0]],['16th', 0, [2], [0]],['16th', 0, [3], [0]],['eighth', 0, [2,3], [0,-1]],['16th', 0, [2], [0]],['16th', 0, [1], [0]],['eighth', 0, [1,2], [0,0]],['16th', 0, [1,3], [0,0]],['16th', 0, [1,2], [0,0]],['eighth', 0, [1,1], [0,1]]]]
tempo_end=[['half', 0, [1,2,3],[0,0,0]],['half', 0, [1,3,1],[0,-1,-1]]]
#structure:duration, dot, chord, octave

pitch = {'C':0, 'D':2, 'E':4, 'F':5, 'G':7, 'A':9, 'B':11}
pitch_inver = {v: k for k, v in pitch.items()}
alter = {'#':1, 'b':-1}
duration_dict = {'whole':1, 'half':1/2, 'quarter':1/4, 'eighth':1/8,\
                 '16th':1/16, '32nd':1/32}
duration_dict_inver = {v: k for k, v in duration_dict.items()}


def creat_measure(number):
    measure_node = ET.Element('measure')
    measure_node.set('number',str(number))
    measure_attributes = ET.SubElement(measure_node, 'attributes')
    measure_attr_divi = ET.SubElement(measure_attributes, 'divisions')
    measure_attr_divi.text = '8'
    measure_attr_key = ET.SubElement(measure_attributes, 'key')
    measure_attr_key_fif = ET.SubElement(measure_attr_key, 'fifths')
    measure_attr_key_fif.text = str(new_fifth)
    measure_attr_time = ET.SubElement(measure_attributes, 'time')
    measure_attr_time_beats = ET.SubElement(measure_attr_time, 'beats')
    measure_attr_time_beats.text = '4'
    measure_attr_time_beats_t = ET.SubElement(measure_attr_time, 'beat-type')
    measure_attr_time_beats_t.text = '4'
    measure_attr_clef = ET.SubElement(measure_attributes, 'clef')
    measure_attr_clef_sign = ET.SubElement(measure_attr_clef, 'sign')
    measure_attr_clef_sign.text = 'G'
    measure_attr_clef_line = ET.SubElement(measure_attr_clef, 'line')
    measure_attr_clef_line.text = '2'
    
    return measure_node

def creat_note(duration, pitch, if_dot, slur):
    note_node = ET.Element('note')
    if pitch == 0:
        rest = ET.SubElement(note_node, 'rest')
    else:
        alter = 0
        try :
            step = pitch_inver[int(pitch)%12]
        
        except KeyError:
            alter+=1
            pitch+=1
            step = pitch_inver[int(pitch)%12]
        
        octave = int(int(pitch)/12)
          
        note_pitch = ET.SubElement(note_node, 'pitch')
        note_pitch_step = ET.SubElement(note_pitch, 'step')
        note_pitch_step.text = str(step)
        note_pitch_octave = ET.SubElement(note_pitch, 'octave')
        note_pitch_octave.text = str(octave)
        if alter == 1:
            note_pitch_alter = ET.SubElement(note_pitch, 'alter')
            note_pitch_alter.text = str(-1)
            
    note_duration = ET.SubElement(note_node, 'duration')
    note_duration.text = str(int(duration_dict[str(duration)])*32)
    note_type = ET.SubElement(note_node, 'type')
    note_type.text = str(duration)
    
    if if_dot==1:
        note_dot = ET.SubElement(note_node, 'dot')
        
    if len(slur)==1:
        note_tie = ET.SubElement(note_node, 'tie')
        note_tie.set('type', slur[0])
        note_notations = ET.SubElement(note_node, 'notations')
        note_notations_tied = ET.SubElement(note_notations, 'tied')
        note_notations_tied.set('type', slur[0])
    
    elif len(slur)==2:
        note_tie = ET.SubElement(note_node, 'tie')
        note_tie.set('type', slur[0])
        note_tie = ET.SubElement(note_node, 'tie')
        note_tie.set('type', slur[1])
        note_notations = ET.SubElement(note_node, 'notations')
        note_notations_tied = ET.SubElement(note_notations, 'tied')
        note_notations_tied.set('type', slur[0])
        note_notations_tied = ET.SubElement(note_notations, 'tied')
        note_notations_tied.set('type', slur[1])

    return note_node
        
def company_measure_creat(new_part,chord_selected, octave_selected, measure_type, note_num):
    number = len(new_part.getchildren())+1
    #company_measure_node = ET.SubElement(new_part, 'measure')
    #company_measure_node.set('number',str(number))
    company_measure_node = creat_measure(number)
    new_part.append(company_measure_node)
    chord_using = chord[chord_selected]
    
    if measure_type=='open':
        company_type = ET.SubElement(company_measure_node, 'company_type')
        company_type.set('type', 'open')
        company_type.set('number', '0')
        for notes in tempo_open: # bar
            for num_chord in range(len(notes[3])): # chord
                print(notes, num_chord, chord_using)
                if num_chord ==0:
                    new_noteinmeasure = creat_note(notes[0], (chord_using[notes[2][num_chord]-1]+(notes[3][num_chord])*12 +octave_selected*12), notes[1], [])
                    company_measure_node.append(new_noteinmeasure)
                else:
                    new_noteinmeasure = creat_note(notes[0], (chord_using[notes[2][num_chord]-1]+(notes[3][num_chord])*12 +octave_selected*12), notes[1], [])
                    tag_chord = ET.Element('chord')
                    new_noteinmeasure.insert(0,tag_chord)
                    company_measure_node.append(new_noteinmeasure)
                
    
    elif measure_type=='run':
        company_type = ET.SubElement(company_measure_node, 'company_type')
        company_type.set('type', 'run')  
        last_company = new_part[len(new_part)-2].find('company_type') #-2 because the last is current one
        
        if last_company.attrib['type']=='run' and last_company.attrib['number']!='0':
            r = int(last_company.attrib['number'])-1
            company_type.set('number', '0')
            for notes in tempo_run[r]: # bar
                for num_chord in range(len(notes[3])): # chord
                    if num_chord ==0:
                        new_noteinmeasure = creat_note(notes[0], (chord_using[notes[2][num_chord]-1]+(notes[3][num_chord])*12 +octave_selected*12), notes[1], [])
                        company_measure_node.append(new_noteinmeasure)
                    else:
                        new_noteinmeasure = creat_note(notes[0], (chord_using[notes[2][num_chord]-1]+(notes[3][num_chord])*12 +octave_selected*12), notes[1], [])
                        tag_chord = ET.Element('chord')
                        new_noteinmeasure.insert(0,tag_chord)
                        company_measure_node.append(new_noteinmeasure)
            
            
        else:
            r= int(random.random()*100)%4
            prob1 = min(pow(1.1, note_num-6),2)
            prob2 = min(pow(1.05, note_num-6),2)
            print(prob1,prob2, note_num)
            if r == 0 or r == 3: # adjust the acomp type by note_num
                if random.random()>(0.5*prob1):
                    r=0
                else:
                    r=3
            if r == 1 or r == 2:
                if random.random()>(0.5*(prob2)):
                    r=1
                else:
                    r=2           
            
            
            company_type.set('number', str(r+1)) # 0 shows that next run should random, 1,2,3 shows the last tempo should be followed
            for notes in tempo_run[r]: # bar
                for num_chord in range(len(notes[3])): # chord
                    if num_chord ==0:
                        new_noteinmeasure = creat_note(notes[0], (chord_using[notes[2][num_chord]-1]+(notes[3][num_chord])*12 +octave_selected*12), notes[1], [])
                        company_measure_node.append(new_noteinmeasure)
                    else:
                        new_noteinmeasure = creat_note(notes[0], (chord_using[notes[2][num_chord]-1]+(notes[3][num_chord])*12 +octave_selected*12), notes[1], [])
                        tag_chord = ET.Element('chord')
                        new_noteinmeasure.insert(0,tag_chord)
                        company_measure_node.append(new_noteinmeasure)
                        
    elif measure_type=='end':
        company_type = ET.SubElement(company_measure_node, 'company_type')
        company_type.set('type', 'end')
        company_type.set('number', '0')
        for notes in tempo_end: # bar
            for num_chord in range(len(notes[3])): # chord
                if num_chord ==0:
                    new_noteinmeasure = creat_note(notes[0], (chord_using[notes[2][num_chord]-1]+(notes[3][num_chord])*12 +octave_selected*12), notes[2], [])
                    company_measure_node.append(new_noteinmeasure)
                else:
                    new_noteinmeasure = creat_note(notes[0], (chord_using[notes[2][num_chord]-1]+(notes[3][num_chord])*12 +octave_selected*12), notes[2], [])
                    tag_chord = ET.Element('chord')
                    new_noteinmeasure.insert(0,tag_chord)
                    company_measure_node.append(new_noteinmeasure)
    
    

    voice_bass1 = ET.SubElement(new_noteinmeasure,'voice')
    voice_bass1.text = '1'
    staff_bass1 = ET.SubElement(new_noteinmeasure, 'staff')
    staff_bass1.text = '1'
    
    backup_comp = ET.SubElement(company_measure_node, 'backup')
    dura_comp = ET.SubElement(backup_comp, 'duration')
    dura_comp.text = '32'
    bass_note = creat_note('whole', (chord_using[0]+(octave_selected-1)*12), 0,[])
    company_measure_node.append(bass_note)
        
    
    voice_bass2 = ET.SubElement(bass_note,'voice')
    voice_bass2.text = '5'
    staff_bass2 = ET.SubElement(bass_note, 'staff')
    staff_bass2.text = '2'
    


def create_stop_measure(new_part):
    number = len(new_part.getchildren())+1
    company_measure_node = ET.SubElement(new_part, 'measure')
    company_measure_node.set('number',str(number))
    stop_note = ET.SubElement(company_measure_node, 'note')
    rest_note = ET.SubElement(stop_note, 'rest')
    dur_note = ET.SubElement(stop_note, 'duration')
    dur_note.text = '32'
    type_note = ET.SubElement(stop_note, 'duration')
    type_note.text = 'whole'    
    
    
        

In [7]:
#add company instrument
comp_score_part = ET.Element('score-part')
comp_score_part.set('id', 'C1')
c_sp_partname = ET.SubElement(comp_score_part, 'part-name')
c_sp_partname.text = '鋼琴'
c_sp_partabbreviation = ET.SubElement(comp_score_part, 'part-abbreviation')
c_sp_partabbreviation.text = 'Pno.'
c_sp_scoreinstrument = ET.SubElement(comp_score_part, 'score-instrument')
c_sp_scoreinstrument.set('id', 'C1-I1')
c_sp_instrumentname = ET.SubElement(c_sp_scoreinstrument, 'instrument-name')
c_sp_instrumentname.text = '鋼琴'
c_sp_mididevice = ET.SubElement(comp_score_part, 'midi-device')
c_sp_mididevice.set('id', 'C1-I1')
c_sp_mididevice.set('port', '1')
c_sp_midiinstrument = ET.SubElement(comp_score_part, 'midi-instrument')
c_sp_midiinstrument.set('id', 'C1-I1')
c_sp_midichannel = ET.SubElement(c_sp_midiinstrument, 'midi-channel')
c_sp_midichannel.text = '2'
c_sp_midiprogram = ET.SubElement(c_sp_midiinstrument, 'midi-program')
c_sp_midiprogram.text = '1'
c_sp_volume = ET.SubElement(c_sp_midiinstrument, 'volume')
c_sp_volume.text = str(new_volume)
c_sp_pan = ET.SubElement(c_sp_midiinstrument, 'pan')
c_sp_pan.text = '0'

new_part_list.append(comp_score_part)





In [8]:
len(new_part1)

28

In [9]:
head_tail= []
for index, i in enumerate(new_part1): #find head and tail
    if_pitch=False
    for j in i:
        if j.find('pitch') is not None:
            if_pitch = True
    if if_pitch and len(head_tail)==0:
        head_tail.append(index)
        head_tail.append(index)
    
    if if_pitch and len(head_tail)==2:
        head_tail[1]=index

for index, i in enumerate(new_part1):
    print(index, len(new_part2))
    measure_comp_list=[]
    if index==head_tail[0]:
        company_type='open'
    elif index==head_tail[1]:
        company_type='end'
    elif index>head_tail[0] and index<head_tail[1]:
        company_type='run'
    else:
        create_stop_measure(new_part2)
        continue
    
    node_list = []
    for j in i.findall('note'):
        
        ta_node = j.find('pitch')
        
        if ta_node is not None:  #extract the notes 
            if ta_node.find('alter') is not None:
                note_pitch = pitch[str(ta_node.find('step').text)] + \
                int(ta_node.find('alter').text) + (int(ta_node.find('octave').text)) *12           

            else:
                note_pitch = pitch[str(ta_node.find('step').text)] + \
                (int(ta_node.find('octave').text)) *12
        
            if j.find('duration') is not None:
                note_duraiton = int(j.find('duration').text)
                
            node_list.append([note_duraiton, note_pitch])
        
    chord_distance=[]
    note_octave=[]
    chord_selected=0
    octave_selected=0
    max_chord=0
    chord_good=[]
    lowest_pitch = 120
    note_num = len(node_list)
    for k in chord: # fit the chord
        temp=0
        for q in node_list:
            if q[1]<lowest_pitch:
                lowest_pitch = q[1]
            note_octave.append(int(q[1]/12))
            if int(q[1]%12) in k or int(q[1]%12)+12 in k:
                temp+=1
        chord_distance.append(temp)
    
    for index, q in enumerate( chord_distance):
        #print(chord_distance)
        #if q> max_chord:
        #    chord_selected=index
        #    max_chord=q
        if q > max_chord:
            chord_good=[index]
            max_chord = q
        elif q == max_chord:
            chord_good.append(index)
    
    
    chord_selected= chord_good[int(random.random()*100)%len(chord_good)]
    octave_selected = max([min(note_octave)-1, 1])
    
    if chord[chord_selected][2]+octave_selected*12+5 >lowest_pitch:
        octave_selected-=1
    if chord[chord_selected][0]+octave_selected*12+24 <lowest_pitch:
        octave_selected+=1
    
    
    company_measure_creat(new_part2, chord_selected, octave_selected, company_type, note_num)
    
    

            
            

0 0
1 1
2 2
['half', 0, [1, 3], [0, 0]] 0 [2, 5, 9]
['half', 0, [1, 3], [0, 0]] 1 [2, 5, 9]
['eighth', 0, [1], [0]] 0 [2, 5, 9]
['quarter', 1, [1, 1], [0, 1]] 0 [2, 5, 9]
['quarter', 1, [1, 1], [0, 1]] 1 [2, 5, 9]
3 3
1.0 1.0 6
4 4
5 5
1.0 1.0 6
6 6
7 7
1.0 1.0 6
8 8
9 9
1.0 1.0 6
10 10
11 11
1.0 1.0 6
12 12
13 13
1.0 1.0 6
14 14
15 15
1.0 1.0 6
16 16
17 17
1.0 1.0 6
18 18
19 19
1.1 1.05 7
20 20
21 21
1.1 1.05 7
22 22
23 23
1.1 1.05 7
24 24
25 25
1.0 1.0 6
26 26
27 27


D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:213: DeprecationWarning: This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:103: DeprecationWarning: This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.


In [10]:
node_list

[[8, 53]]

In [11]:
def indent(elem, level=0):
    i = "\n" + level*"\t"
    if len(elem):
        if not elem.text or not elem.text.strip():
            elem.text = i + "\t"
        if not elem.tail or not elem.tail.strip():
            elem.tail = i
        for elem in elem:
            indent(elem, level+1)
        if not elem.tail or not elem.tail.strip():
            elem.tail = i
    else:
        if level and (not elem.tail or not elem.tail.strip()):
            elem.tail = i

In [12]:


    indent(new_root)
    tree = ET.ElementTree(new_root)
    tree.write(output_file)

In [13]:
for i in new_part.find('measure')[1]:
    print(i)

NameError: name 'new_part' is not defined

In [ ]:
i = [1,2,5]
max(i)

In [ ]:
if 1 in [2,3] or 1 in [2,3]:
    print('ooo')